# 0. Importación de librerías y archivos

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium

# 1. Análisis exploratorio inicial

Importar archivos base

In [ ]:
base = pd.read_csv('datasets/dengue.csv')

## 1.1. Visualización espacial

Ahora que disponemos que la información poligonal tanto para los distritos y las provincias, combinaremos ambas tablas para poder realizar una exploración espacial.

In [ ]:
# Combinación con las tablas de información geográfica poligonal
base.set_index(['año','provincia','distrito'],inplace=True)

base_dist = base.groupby(level=[0,1,2])[['casos','tia']].sum().reset_index()
base_prov = base.groupby(level=[0,1])[['casos','tia']].sum().reset_index()

# Combinación con las tablas de información geográfica poligonal
geoprov = gpd.GeoDataFrame(base_prov.merge(provincial,how='left',on='provincia')).set_geometry('geometry')
geodist = gpd.GeoDataFrame(base_dist.merge(distrital,how='left',on=['provincia','distrito'])).set_geometry('geometry')

# Revisando que las tablas generadas tengan la totalidad de registros
print(f'Completitud de datos en distritos (filas totales: {geodist.shape[0]}): \n{geodist.count()}')
geodistaño = geodist[(geodist['año']==2024)& ~(geodist['objectid'].isna())]

Completitud de datos en distritos (filas totales: 890): 
año          890
provincia    890
distrito     890
casos        890
tia          890
objectid     880
geometry     880
dtype: int64


In [ ]:
m = folium.Map(location=[geodistaño.centroid.y.mean(), geodistaño.centroid.x.mean()], zoom_start=7.2)
range = (geodistaño['casos'].quantile((0,0.3,0.5,0.8,0.9,1))).tolist()
cp = folium.Choropleth(
        geo_data=geodistaño,
        name="choropleth",
        data=geodist,
        columns=['objectid','casos'],
        key_on='feature.properties.objectid',
        threshold_scale=range,
        fill_color="YlGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="Incidencias de dengue 2024 (TIA)",
        smooth_factor=0
).add_to(m)


fields=['provincia','distrito','casos']
aliases=['Provincia: ','Distrito: ','Casos: ']
        
funk = folium.features.GeoJson(
    geodistaño,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=fields,
        aliases=aliases,
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(funk)
m.keep_in_front(funk)
folium.LayerControl().add_to(m)
        
display(m)